In [2]:
import pandas as pd
import requests

In [3]:
def extract_data_from_nasa_api(url, max_pages):
    """Получает JSON-данные по API"""
    all_objects = []  # Список для хранения всех астероидов
    current_url = url
    page = 1

    while current_url and page <= max_pages:
        try:
            response = requests.get(current_url)
            response.raise_for_status()
            data = response.json()
            
    # Добавляем астероиды с текущей страницы
            all_objects.extend(data.get("near_earth_objects", []))
        
    # Проверяем, есть ли следующая страница
            current_url = data["links"].get("next")
            print(f"Загружена страница {page}")
            page += 1
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса к API: {e}")
            break
        
    return all_objects  # Возвращаем список всех астероидов  

In [4]:
def get_near_earth_objects_data(data):
    """Извлекает список астероидов в DataFrame"""
    try:
        if not data:
            print('Данные об астероидах отсутствуют')
            return pd.DataFrame()
        near_earth_objects = pd.json_normalize(data)
        print("Данные об астероидах добавлены")
        return near_earth_objects
    except Exception as e:
        print(f"Ошибка при обработке данных об астероидах: {e}")

In [5]:
def get_close_approaches(data):
    """Извлекает данные о сближениях астероидов"""
    try:
        close_approach = pd.json_normalize(data, 'close_approach_data', ['id', 'name'], errors="ignore")
        print("Данные о сближениях загружены")
        return close_approach
    except Exception as e:
        print(f"Ошибка при обработке данных о сближениях астероидов: {e}")
        return None

In [6]:
def write_data_to_excel(near_earth_objects, close_approach, filename):
    """Сохраняет данные в один Excel-файл с разными листами"""
    try:
        with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
            near_earth_objects.to_excel(writer, sheet_name="Asteroids", index=False)
            if close_approach is not None:
                close_approach.to_excel(writer, sheet_name="Close_Approaches", index=False)
        print(f"Данные успешно сохранены в '{filename}'")
    except Exception as e:
        print(f"Ошибка записи данных в Excel: {e}")


In [7]:
def main(api_key, base_url):
    """Основная логика программы"""
    url = f"{base_url}?api_key={api_key}"
    data = extract_data_from_nasa_api(url, max_pages=10) # Взяла первые 10 страниц

    asteroids_df = get_near_earth_objects_data(data)
    close_approaches_df = get_close_approaches(data)
    write_data_to_excel(asteroids_df, close_approaches_df, 'files/nasa_neo_data.xlsx')


if __name__ == "__main__":
    base_url = "502SJWBgC67LUsXow8S2uPQtZC29d2iDQ7Og6Hkg"
    api_key = "https://api.nasa.gov/neo/rest/v1/neo/browse"
    main(base_url, api_key)

Загружена страница 1
Загружена страница 2
Загружена страница 3
Загружена страница 4
Загружена страница 5
Загружена страница 6
Загружена страница 7
Загружена страница 8
Загружена страница 9
Загружена страница 10
Данные об астероидах добавлены
Данные о сближениях загружены
Данные успешно сохранены в 'files/nasa_neo_data.xlsx'
